<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Scenario-1" data-toc-modified-id="Scenario-1-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scenario 1</a></span><ul class="toc-item"><li><span><a href="#Build-Graph" data-toc-modified-id="Build-Graph-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Build Graph</a></span></li></ul></li></ul></div>

# Description

This notebook aims at testing the two main algorithms on larger graphs in order to test for the scalability of the approach

# Load

In [1]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from routines_icu import *
from helpers_icu import *
import cvxpy as cp
from FW_icu import *

from FW_OuterUpdate import solve

# Scenario 1

I want to test on three nodes, first. A fully connected graph with three nodes

In [6]:
import pandas as pd
import os

In [21]:
path='Data/2Nodes/'

In [67]:
edges=pd.read_excel(os.path.join(path,'edges.xlsx'))

In [68]:
edges

,head,tail,length,capacity,time,is negative
0,1,2,10,10,10,0
1,2,1,10,10,10,0


In [75]:
#convert into strings for the first two columns
for i in range(edges.shape[0]):
    for col in ['head', 'tail']:
        edges.loc[i,col]=str(edges.loc[i,col])

In [58]:
data=np.array([['1','R', 0,0,0,1]])

In [59]:
data.shape

(1, 6)

In [60]:
l=pd.DataFrame(columns=edges.columns,data=data)

In [61]:
df=edges.append(l,ignore_index=True)

In [62]:
df

,head,tail,length,capacity,time,is negative
0,1,2,10,10,10,0
1,2,1,10,10,10,0
2,1,R,0,0,0,1


In [91]:
def expand_graph(edges,OD):
    
    edges=edges.copy()
    
    
    #######################################
    # 1 . Add rebalancing edge
    
    #what really matters is the value of phi
    L_rebalancing_edge=10
    t_rebalancing_edge=10
    k_rebalancing_edge=1 #does not matter as it should be adapted accordingly
    
    for (o,_) in OD.keys(): #loop over the different origins in OD pairs
        data=np.array([[str(o),'R', L_rebalancing_edge,
                       k_rebalancing_edge,t_rebalancing_edge,0]])
        l=pd.DataFrame(columns=edges.columns,data=data)
        edges=edges.append(l,ignore_index=True)
        
    return edges
    

In [92]:
#Note: it is the same to write OD[(a,b)] as OD[a,b]

OD=dict()
N=10
OD['1','2_p']=N
OD['2','1_p']=N

In [93]:
edges=expand_graph(edges,OD)

In [94]:
edges

,head,tail,length,capacity,time,is negative
0,1,2,10,10,10,0
1,2,1,10,10,10,0
2,1,R,10,1,10,0
3,2,R,10,1,10,0
4,1,R,10,1,10,0
5,2,R,10,1,10,0
6,1,R,10,1,10,0
7,2,R,10,1,10,0


## Build Graph

In [2]:
#parameters
alpha=0.15
beta=4
POTENTIAL_SHIFT=90
INVERSE_DEMAND_SHIFT=80

ZC_EDGE=1

In [ ]:
L_dum=10
#to adapt
#to be read from file if necessary
nodes_list=['1','2','2_p','R','1_p']
edge_list=[('1','2'),('2','1'),('2','2_p'),('1','R'),('1','2_p'),('1','1_p'),('2','1_p'),('2','R')]
k_list=[10,10,3,10,ZC_EDGE,2,ZC_EDGE,10] 
l_list=[10,10,1,L_dum,0,1,0,L_dum]
t_list=[10,10,10,10,ZC_EDGE,10,ZC_EDGE,10]
phi_list=[phi(l_list[i],t_list[i]) for i in range(len(l_list))]
is_negative=[0,0,1,0,0,1,0,0]

nodes_pots=[('2_p',POTENTIAL_SHIFT),('1_p',POTENTIAL_SHIFT)]

In [ ]:
dummy_nodes=dict()
dummy_nodes['2_p']='2'
dummy_nodes['1_p']='1'